In [36]:
import pandas as pd
import numpy as np
import os

In [ ]:
current_dir = os.getcwd()
url_wti_crude_oil = os.path.join(current_dir, "../data/raw/macrodata/wti_crude_oil.csv")


## Brent Oil

In [ ]:
wti_oil_raw = pd.read_csv(url_wti_crude_oil, sep = ",")
wti_oil_raw

,Date,Open,High,Low,Close,Adj Close,Volume
0,"May 8, 2018",75.66,76.18,73.10,74.85,74.85,"50,813"
1,"May 7, 2018",74.98,76.33,74.85,76.17,76.17,"21,266"
2,"May 4, 2018",73.76,75.05,73.25,74.87,74.87,"30,457"
3,"May 3, 2018",73.14,74.08,72.48,73.62,73.62,"35,839"
4,"May 2, 2018",73.21,73.62,72.37,73.36,73.36,"34,769"
...,...,...,...,...,...,...,...
83,"Jan 8, 2018",67.82,67.98,67.45,67.78,67.78,"17,704"
84,"Jan 5, 2018",68.06,68.09,67.27,67.62,67.62,"18,290"
85,"Jan 4, 2018",68.00,68.26,67.53,68.07,68.07,"22,840"
86,"Jan 3, 2018",66.49,68.01,66.40,67.84,67.84,"25,878"


In [ ]:
wti_oil = wti_oil_raw[["Date", "Close"]]
wti_oil

,Date,Close
0,"May 8, 2018",74.85
1,"May 7, 2018",76.17
2,"May 4, 2018",74.87
3,"May 3, 2018",73.62
4,"May 2, 2018",73.36
...,...,...
83,"Jan 8, 2018",67.78
84,"Jan 5, 2018",67.62
85,"Jan 4, 2018",68.07
86,"Jan 3, 2018",67.84


## DATA

In [63]:
url_sales = os.path.join(current_dir, "../data/raw/sales.csv")
sales = pd.read_csv(url_sales, sep=",")

In [66]:
sales

,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3
...,...,...,...,...,...,...,...,...,...
6758120,6758121,10,50938,300,13,0.0,0.0,2018-04-20 22:39:03.150,AHVBHVO7632V5YYD2VJT
6758121,6758122,18,85668,452,22,0.1,0.0,2018-01-13 15:08:18.700,M4IA0FS13J9F0GJD4BX8
6758122,6758123,21,68409,405,18,0.0,0.0,2018-01-18 12:05:16.950,7VZJ77UGBSX2SNCC1GFW
6758123,6758124,17,90315,104,23,0.0,0.0,2018-02-26 08:13:37.850,3JTF5IF0YJLLXF9IRS8X


In [65]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6758125 entries, 0 to 6758124
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   SalesID            int64  
 1   SalesPersonID      int64  
 2   CustomerID         int64  
 3   ProductID          int64  
 4   Quantity           int64  
 5   Discount           float64
 6   TotalPrice         float64
 7   SalesDate          object 
 8   TransactionNumber  object 
dtypes: float64(2), int64(5), object(2)
memory usage: 464.0+ MB
